# Imports

In [42]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Get the dataset

---
We are going to use a dataset containing amazon and yelp reviews


In [43]:
!wget --no-check-certificate \
    -O /tmp/sentiment.csv https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P

--2021-04-03 17:57:41--  https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving drive.google.com (drive.google.com)... 172.217.218.113, 172.217.218.139, 172.217.218.138, ...
Connecting to drive.google.com (drive.google.com)|172.217.218.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/slrmf2a91t7qo97hrqeo1ln8vkd3gbug/1617472650000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P [following]
--2021-04-03 17:57:41--  https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/slrmf2a91t7qo97hrqeo1ln8vkd3gbug/1617472650000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)... 108.177.126.132, 2a00:1450:4013:c01::84
Connecting to doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)|1

In [44]:
dataset = pd.read_csv('/tmp/sentiment.csv')

sentences = dataset['text'].tolist()
labels = dataset['sentiment'].tolist()

# Separate out the sentences and labels into training and test sets
training_size = int(len(sentences) * 0.8)

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

# Make labels into numpy arrays for use with the network later
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

# Tokenizing the data


In [45]:
vocab_size = 1000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, padding=padding_type, 
                       truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length, 
                               padding=padding_type, truncating=trunc_type)

# Review a sequence

In [46]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))

good case excellent value ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?


In [47]:
print(train_sentences[1])

Good case Excellent value.


# Time to train a basic model with embeddings

In [48]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 16)           16000     
_________________________________________________________________
flatten_5 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 9606      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 7         
Total params: 25,613
Trainable params: 25,613
Non-trainable params: 0
_________________________________________________________________


In [49]:
num_epochs = 10
model.fit(padded, 
          training_labels_final, 
          epochs=num_epochs,
          validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
50/50 [==============================] - 1s 6ms/step - loss: 0.6926 - accuracy: 0.5279 - val_loss: 0.6971 - val_accuracy: 0.4110
Epoch 2/10
50/50 [==============================] - 0s 3ms/step - loss: 0.6875 - accuracy: 0.5162 - val_loss: 0.7014 - val_accuracy: 0.4110
Epoch 3/10
50/50 [==============================] - 0s 4ms/step - loss: 0.6730 - accuracy: 0.5212 - val_loss: 0.6894 - val_accuracy: 0.4486
Epoch 4/10
50/50 [==============================] - 0s 3ms/step - loss: 0.6395 - accuracy: 0.6362 - val_loss: 0.6805 - val_accuracy: 0.4637
Epoch 5/10
50/50 [==============================] - 0s 4ms/step - loss: 0.5767 - accuracy: 0.6919 - val_loss: 0.6614 - val_accuracy: 0.5238
Epoch 6/10
50/50 [==============================] - 0s 4ms/step - loss: 0.5185 - accuracy: 0.8099 - val_loss: 0.6257 - val_accuracy: 0.6216
Epoch 7/10
50/50 [==============================] - 0s 4ms/step - loss: 0.4128 - accuracy: 0.9141 - val_loss: 0.4976 - val_accuracy: 0.7694
Epoch 8/10
50/50 [==

# Get files for visualasing the network

In [50]:
e = model.layers[0]
weights = e.get_weights()[0]

print(weights.shape)

(1000, 16)


In [51]:
import io

# Write out the embedding vectors and metadata
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [52]:
# Use the model to predict a review   
fake_reviews = ['I love this phone', 'I hate suck spaghetti', 
                'Everything was cold',
                'Everything was hot exactly as I wanted', 
                'Everything was green', 
                'the host seated us immediately',
                'they gave us free chocolate cake', 
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes', 
                'does not work when I stand on my head']

print(fake_reviews) 

# Create the sequences
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)
fakes_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=max_length)           

print('\nHOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!\n')              

classes = model.predict(fakes_padded)

# The closer the class is to 1, the more positive the review is deemed to be
for x in range(len(fake_reviews)):
  print(fake_reviews[x])
  print(classes[x])
  print('\n')


['I love this phone', 'I hate suck spaghetti', 'Everything was cold', 'Everything was hot exactly as I wanted', 'Everything was green', 'the host seated us immediately', 'they gave us free chocolate cake', 'not sure about the wilted flowers on the table', 'only works when I stand on tippy toes', 'does not work when I stand on my head']

HOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!

I love this phone
[0.9885554]


I hate suck spaghetti
[0.07216766]


Everything was cold
[0.2059781]


Everything was hot exactly as I wanted
[0.5428037]


Everything was green
[0.34160107]


the host seated us immediately
[0.6895029]


they gave us free chocolate cake
[0.82868683]


not sure about the wilted flowers on the table
[0.02186109]


only works when I stand on tippy toes
[0.9243843]


does not work when I stand on my head
[0.01065897]


